# i2i queries

### Number of total points in each type, sector and country

```sql
SELECT count(type), type, sector, iso   
    FROM fsp_maps
   WHERE iso = 'KEN'
GROUP BY sector, iso, type
```

### Coordinates selected points

```sql
SELECT the_geom, the_geom_webmercator, type, sector
    FROM fsp_maps
   WHERE iso = 'KEN' AND sector in ('Agriculture', 'Finance') AND type in ('Agro-Dealers', 'Market Locations', 'Mobile Money Agent', 'Bank Agent', 'Forex Bureaus')
GROUP BY sector, type
```

```sql
SELECT the_geom, the_geom_webmercator
FROM fsp_maps
WHERE iso = 'KEN' and sector = 'Agriculture' and type = 'Agro-Dealers'
```

### Coordinates selected points' voronoids

```sql
WITH v as (SELECT v.*, m.type
             FROM fsp_voronoid as v
        LEFT JOIN fsp_maps as m
               ON v.id = m.id 
         ORDER BY v.id)
            
SELECT the_geom, the_geom_webmercator
FROM v
WHERE iso = 'KEN' and sector = 'Agriculture' and type = 'Agro-Dealers'
```

### Population inside a country
** Total, urban, and rural population, their percentages, and population within 5 km of all access points**

```sql
WITH b as (SELECT the_geom FROM world_borders_hd as b 
            WHERE b.iso_a3 = 'KEN'),
     u as (SELECT u.the_geom FROM urban_areas as u
             JOIN world_borders_hd as b
               ON ST_Intersects(b.the_geom, u.the_geom)
            WHERE b.iso_a3 = 'KEN')

SELECT tot.pop as total_population, urb.pop as urban_population, 
  round(((urb.pop/tot.pop)*100)::numeric, 1) as urban_population_percentage, (tot.pop - urb.pop) as 
  rural_population, round((((tot.pop - urb.pop)/tot.pop)*100)::numeric,1) as   
  rural_population_percentage, round(pop_km.tot_pop_km::numeric) as population_5km,
  round(((round(pop_km.tot_pop_km::numeric)/tot.pop)*100)::numeric, 1) as population_5km_percentage,
  2015 as year
FROM (SELECT round((ST_SummaryStatsAgg(ST_Clip(the_raster_webmercator, b.the_geom),1,True)).sum) as pop
        FROM population_count_2015, b 
       WHERE ST_Intersects(the_raster_webmercator, b.the_geom)
      ) as tot,
     (SELECT round((ST_SummaryStatsAgg(ST_Clip(the_raster_webmercator, u.the_geom),1,True)).sum) as pop
        FROM population_count_2015, u 
       WHERE ST_Intersects(the_raster_webmercator, u.the_geom)
      ) as urb,
     (SELECT total_population as tot_pop_km
        FROM population_within_5km 
       WHERE iso = 'KEN'
      ) as pop_km
```

### Total population within 5 km of all access points in a country

```sql
WITH buff as (SELECT ST_Union(ST_Buffer(the_geom_webmercator,5000)) as the_geom_webmercator
                FROM fsp_maps WHERE iso = 'KEN'),
        b as (SELECT the_geom FROM world_borders_hd as b 
               WHERE b.iso_a3 = 'KEN'),
   buff_b as (SELECT ST_Intersection(b.the_geom, st_transform(buff.the_geom_webmercator,4326)) as the_geom
                FROM b, buff)

SELECT pop_5km.pop as total_population_within_5km, 
round(((pop_5km.pop/tot.pop)*100)::numeric, 1) as total_population_within_5km_percentage, 'KEN' as iso
FROM (SELECT round((ST_SummaryStatsAgg(ST_Clip(the_raster_webmercator,  buff_b.the_geom),1,True)).sum) as pop
        FROM population_count_2015, buff_b
       WHERE ST_Intersects(the_raster_webmercator, buff_b.the_geom)
     ) as pop_5km,
    (SELECT round((ST_SummaryStatsAgg(ST_Clip(the_raster_webmercator,b.the_geom),1,True)).sum) as pop
       FROM population_count_2015, b 
      WHERE ST_Intersects(the_raster_webmercator, b.the_geom)
     ) as tot
```

### Total selected access points and population 

** Inside a country **

```sql
WITH b as (SELECT the_geom FROM world_borders_hd as b 
            WHERE b.iso_a3 = 'KEN'),
     u as (SELECT u.the_geom FROM urban_areas as u
             JOIN world_borders_hd as b
               ON ST_Intersects(b.the_geom, u.the_geom)
            WHERE b.iso_a3 = 'KEN')

SELECT points.count as access_points, tot.pop as total_population, urb.pop as urban_population, 
  round(((urb.pop/tot.pop)*100)::numeric, 1) as urban_population_percentage, (tot.pop - urb.pop) as 
  rural_population, round((((tot.pop - urb.pop)/tot.pop)*100)::numeric,1) as   
  rural_population_percentage
FROM (SELECT round((ST_SummaryStatsAgg(ST_Clip(the_raster_webmercator, b.the_geom),1,True)).sum) as pop
        FROM population_count_2015, b 
       WHERE ST_Intersects(the_raster_webmercator, b.the_geom)
      ) as tot,
     (SELECT round((ST_SummaryStatsAgg(ST_Clip(the_raster_webmercator, u.the_geom),1,True)).sum) as pop
        FROM population_count_2015, u 
       WHERE ST_Intersects(the_raster_webmercator, u.the_geom)
      ) as urb,
    (SELECT count(*)
        FROM fsp_maps
       WHERE iso = 'KEN' AND sector in ('Agriculture', 'Finance') AND type in ('Agro-Dealers', 'Market Locations', 'Mobile Money Agent', 'Bank Agent', 'Forex Bureaus')
     ) as points
```

** Inside a geometry **

```sql
WITH b as (SELECT the_geom FROM world_borders_hd as b 
            WHERE b.iso_a3 = 'KEN'),
     u as (SELECT u.the_geom FROM urban_areas as u
             JOIN world_borders_hd as b
               ON ST_Intersects(b.the_geom, u.the_geom)
            WHERE b.iso_a3 = 'KEN')

SELECT points.count as access_points, tot.pop as total_population, urb.pop as urban_population, 
  round(((urb.pop/tot.pop)*100)::numeric, 1) as urban_population_percentage, (tot.pop - urb.pop) as 
  rural_population, round((((tot.pop - urb.pop)/tot.pop)*100)::numeric,1) as   
  rural_population_percentage
FROM (SELECT round((ST_SummaryStatsAgg(ST_Clip(the_raster_webmercator, b.the_geom),1,True)).sum) as pop
        FROM population_count_2015, b 
       WHERE ST_Intersects(the_raster_webmercator, b.the_geom)
      ) as tot,
     (SELECT round((ST_SummaryStatsAgg(ST_Clip(the_raster_webmercator, u.the_geom),1,True)).sum) as pop
        FROM population_count_2015, u 
       WHERE ST_Intersects(the_raster_webmercator, u.the_geom)
      ) as urb,
    (SELECT count(*)
        FROM fsp_maps as m, b 
       WHERE ST_Intersects(m.the_geom, b.the_geom) AND m.sector in ('Agriculture', 'Finance') AND m.type in ('Agro-Dealers', 'Market Locations', 'Mobile Money Agent', 'Bank Agent', 'Forex Bureaus')
     ) as points
```

### Access points percentage and number of services

** Inside a country **

```sql
SELECT points.count as access_points, round((points.count*1./tot.count)*100, 1) as access_points_percentage, points.type , points.sector
FROM (SELECT count(*), sector, type
        FROM fsp_maps 
       WHERE iso = 'KEN' AND sector in ('Agriculture', 'Finance') AND type in ('Agro-Dealers', 'Market Locations', 'Mobile Money Agent', 'Bank Agent', 'Forex Bureaus')
    GROUP BY sector, type
     ) as points,
    (SELECT count(*)
        FROM fsp_maps 
       WHERE iso = 'KEN' AND sector in ('Agriculture', 'Finance') AND type in ('Agro-Dealers', 'Market Locations', 'Mobile Money Agent', 'Bank Agent', 'Forex Bureaus')
     ) as tot
```

** Inside a geometry **

```sql
WITH b as (SELECT the_geom 
             FROM world_borders_hd as b
            WHERE b.iso_a3 = 'KEN')
            
SELECT points.count as access_points, round((points.count*1./tot.count)*100, 1) as access_points_percentage, points.type , points.sector
FROM (SELECT count(*), sector, type
        FROM fsp_maps as m, b 
       WHERE ST_Intersects(b.the_geom, m.the_geom) AND m.iso = 'KEN' AND m.sector in ('Agriculture', 'Finance') AND m.type in ('Agro-Dealers', 'Market Locations', 'Mobile Money Agent', 'Bank Agent', 'Forex Bureaus')
    GROUP BY m.sector, m.type
     ) as points,
    (SELECT count(*)
        FROM fsp_maps as m, b 
       WHERE ST_Intersects(b.the_geom, m.the_geom) AND m.iso = 'KEN' AND m.sector in ('Agriculture', 'Finance') AND m.type in ('Agro-Dealers', 'Market Locations', 'Mobile Money Agent', 'Bank Agent', 'Forex Bureaus')
     ) as tot
```

### Time distance to access points